# AlphaFold 3 for Predicting Biomolecular Structures and Interactions

**Author**: Xiping Gong (xipinggong@uga.edu, Department of Crop and Soil Sciences, College of Agricultural and Environmental Sciences, University of Georgia, Griffin, GA 30223, Untied States)

**11/6/2025 (first draft)**: This tutorial is initially prepared for the MIBO 8110 course to guide students in learning how to use AlphaFold 3 (AF3) for predicting biomolecular structures and interactions. It provides step-by-step instructions and practical examples to help students understand how AF3 integrates protein and small-molecule components into unified structure prediction workflows. The goal is to help students gain hands-on experience with computational structure prediction tools and appreciate their applications in molecular biology and bioinformatics research.

**2/10/2026**: This tutorial is improved for the beginners who want to use AF3.

# Prerequisites

+ **AF3 is already installed and ready to use.**

If not, you can install it by following the official setup instructions provided on DeepMind’s GitHub repository:

🔗 Official AlphaFold 3 GitHub Repository: https://github.com/google-deepmind/alphafold3

Please refer to the installation section in the repository for detailed instructions on software dependencies, environment setup, and system requirements (e.g., GPU support, Python version, and data model downloads). Once the installation is complete, you can proceed with the following exercises to predict biomolecular structures and interactions.

For the MIBO 8110 students, the AlphaFold 3 has been installed in the teaching node, please log in your account (MyID@teach.gacrc.uga.edu). 

+ **Some Python packages are required to install.**

To make things easier, using the Python scripts can help to run the AF3 jobs. I therefore prepared some Python scripts in the script folder, you need to install some Python packages, like mdtraj and openmm, etc. Otherwise, you could not run these scripts correctly.


# Protein structure predictions

Here are the steps to use AF3 to predict the protein structure from the sequence. We can go to the test folder first, and then practice in this folder. 

```bash
$ cd test # test is a folder for testing.
```

I will use some Python scripts, like download_pdb.py, extract_pdb.py, and cif2pdb.py. To see how to use them, you can just type like this:

```bash
$ python ../scripts/download_pdb.py -h # see the help
```

+ **Step 1. Obtain a native PDB file**
```bash
# Taking the human serium albumin as an example (PDBID: 7AAI).
$ python ../scripts/download_pdb.py 7AAI # A 7AAI.pdb file will be created.

# We can also have a quick check of this pdb file
$ python ../scripts/check_pdb.py 7AAI.pdb # it will list some info of this pdb file.

# The downloaded 7AAI.pdb file not only has the protein, but also includes the other non-protein structures. Here, we will extract the protein structure.
$ python ../scripts/extract_pdb.py --ref 7AAI.pdb --selection "protein" -o 7AAI_protein.pdb # A 7AAI_protein.pdb will be created.
$ python ../scripts/check_pdb.py 7AAI_protein.pdb # you can also have a check of this extracted pdb file.
```
+ **Step 2. Obtain a json file**
```bash
# To run the AF3, it needs the protein sequence, so we will extract it from the created protein pdb structure.
$ python ../scripts/get_json_for_af3.py 7AAI_protein.pdb -o 7AAI_protein.json # the 7AAI_protein.json will be quickly created, please have a look at this json file. This script cannot cover all cases, so you need to have a check and can type it by yourself, and this json file must follow the input requirement (Link: https://github.com/google-deepmind/alphafold3/blob/main/docs/input.md).
```
+ **Step 3. Run an AF3 job**

We will use a bash script to submit the AF3 job, and a template (af3.sh) can be found in the scripts folder, which can be copied to the current directory.

```bash
$ cp ../scripts/af3.sh . # copy the af3.sh, and '.' represents the current directory.
# Please have a look at this af3.sh script, you should modify it before running, like
# #SBATCH --partition=gpu_p             # GPU Partition (queue) name (gpu_p is for UGA Sapelo2). It specifies the GPU node.
# af3_param_dir='/home/xg69107/program/alphafold3' # You have to request the parameters file to run AF3 from Google DeepMind.
# # In the UGA Sapelo2 (this tells where you installed AF3)
# echo "Running af3 model >>" # Check where to install and how to use
# singularity exec \
#      --nv \
#      --bind $work_dir:/root/af_input \
#      --bind $work_dir:/root/af_output \
#      --bind $af3_param_dir:/root/models \
#      --bind /db/AlphaFold3/20241114:/root/public_databases \
#      /apps/singularity-images/alphafold-3.0.0-CCDpatched.sif \
#      python /app/alphafold/run_alphafold.py \
#      --json_path=/root/af_input/$json_file \
#      --model_dir=/root/models \
#      --db_dir=/root/public_databases \
#      --output_dir=/root/af_output
```

```bash
# I assumed that we modified this bash script to run the AF3 job, see af3.sh. Then, we can have it submitted.
$ sbatch af3.sh 7AAI_protein.json # it will run ~10 mins, and a "7aai_protein" folder will be created, which includes the predicted 3D structures.
```
+ **Step 4. Do the post-analysis**
```bash
# To see the predicted structures, severeal extra steps are often needed, like convert the cif to pdb, align the pdb with the native one, etc.
$ python ../scripts/cif2pdb.py "7aai_protein/*/model.cif" # convert model.cif to model.pdb
$ python ../scripts/align_pdb.py --ref 7AAI_protein.pdb "7aai_protein/*/model.pdb" -o 7AAI_protein_af3.pdb # Align all predicted pdb files into one pdb file, in terms of a native pdb structure. The output will be like this:
```
```text
📂 Loading reference: 7AAI_protein.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7AAI_protein.pdb
<mdtraj.Trajectory with 1 frames, 4626 atoms, 581 residues, and unitcells>
📂 Loading target: 7aai_protein/*/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein/seed-1_sample-0/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein/seed-1_sample-1/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein/seed-1_sample-2/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein/seed-1_sample-3/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein/seed-1_sample-4/model.pdb
<mdtraj.Trajectory with 5 frames, 4627 atoms, 581 residues, without unitcells>
🔍 Identifying protein backbone pocket atoms from reference...
⚠ Warning: No ligand found. Using protein backbone heavy atoms as pocket.
📐 Aligning target to reference using protein backbone pocket atoms...
📊 Pocket-Aligned Protein Backbone RMSD (MDTraj): [0.44265428 0.42144    0.42826456 0.42225283 0.41918656] nm
📊 Pocket-Aligned Protein Backbone Pocket RMSD (MDTraj): [0.44265428 0.42144    0.42826456 0.42225283 0.41918656] nm
💾 Saving aligned model structures
 - 7AAI_protein_af3.pdb
ℹ️ No ligand found — skipping ligand output.
✅ Alignment complete.
```

+ **Step 5. Using VMD to see the difference**

You can download these pdb files (7AAI_protein.pdb, 7AAI_protein_af3.pdb), and use the VMD to visualize their structrues.


# Protein-ligand interaction predictions

In this section, we will use the AF3 to predict how the ligand(s) can interact with a protein, and see their interactions. The process is similar to the protein structure prediction, but we need to modify the input json file, like we should include the ligand info.

Similarly, we can **obtain a reference pdb file**, which has one protein and the ligands we need. 
```bash
$ python ../scripts/extract_pdb.py --ref 7AAI.pdb --selection "protein or resname '8PF' or resname 'MYR' " -o 7AAI_protein_ligands.pdb
$ python ../scripts/check_pdb.py 7AAI_protein_ligands.pdb
```

Taking the 7AAI as an example, we can use the check_pdb.py script to check, it has one protein chain, and multiple ligands. We can also use the VMD to see where they are located. 

```text
7AAI_protein_ligands.pdb: <mdtraj.Trajectory with 1 frames, 4790 atoms, 589 residues, and unitcells>

Protein Info:
  - Chain 'A':
    - Number of residues: 581
    - Number of atoms: 4626
  - 7AAI_protein_ligands.pdb: Multiple protein chains present: False

Ligand Info (including water):
  - Ligand '8PF':
    - Chain IDs: ['B', 'C', 'D', 'E']
    - Number of residues: 4
    - Number of atoms: 100
    - 7AAI_protein_ligands.pdb: Multiple '8PF' ligand residues present: True
  - Ligand 'MYR':
    - Chain IDs: ['F', 'G', 'H', 'I']
    - Number of residues: 4
    - Number of atoms: 64
```
Here, we select the protein sequence, 4 '8PF' ligands, and 4 'MYR' ligands. Then, we can **create an input json file** like below.

```json
{
  "name": "7AAI_protein_ligands",
  "sequences": [
    {
      "protein": {
        "id": "A",
        "sequence": "HKSEVAHRFKDLGEENFKALVLIAFAQYLQQCPFEDHVKLVNEVTEFAKTCVADESAENCDKSLHTLFGDKLCTVATLRETYGEMADCCAKQEPERNECFLQHKDDNPNLPRLVRPEVDVMCTAFHDNEETFLKKYLYEIARRHPYFYAPELLFFAKRYKAAFTECCQAADKAACLLPKLDELRDEGKASSAKQRLKCASLQKFGERAFKAWAVARLSQRFPKAEFAEVSKLVTDLTKVHTECCHGDLLECADDRADLAKYICENQDSISSKLKECCEKPLLEKSHCIAEVENDEMPADLPSLAADFVESKDVCKNYAEAKDVFLGMFLYEYARRHPDYSVVLLLRLAKTYETTLEKCCAAADPHECYAKVFDEFKPLVEEPQNLIKQNCELFEQLGEYKFQNALLVRYTKKVPQVSTPTLVEVSRNLGKVGSKCCKHPEAKRMPCAEDYLSVVLNQLCVLHEKTPVSDRVTKCCTESLVNRRPCFSALEVDETYVPKEFNAETFTFHADICTLSEKERQIKKQTALVELVKHKPKATKEQLKAVMDDFAAFVEKCCKADDKETCFAEEGKKLVAASQAAL"
      }
    },
    {
     "ligand": {
        "id": ["B","C","D","E"],
        "ccdCodes": ["8PF"]
      }
    },
    {
     "ligand": {
        "id": ["I","J","K","L"],
        "ccdCodes": ["MYR"]
      }
    }
  ],
  "modelSeeds": [
    1
  ],
  "bondedAtomPairs": [],
  "dialect": "alphafold3",
  "version": 2
}

```

We can also use the script to do,
```bash
$ python ../scripts/get_json_for_af3.py 7AAI_protein_ligands.pdb -o 7AAI_protein_ligands.json
```

Then, we can **run an AF3 job**,
```bash
$ sbatch af3.sh 7AAI_protein_ligands.json # it will run ~10 mins, and a "7aai_protein_ligands" folder will be created, which includes the predicted 3D complex structures.
```

+ **Do the post-analysis**
```bash
$ python ../scripts/cif2pdb.py "7aai_protein_ligands/*/model.cif" # convert model.cif to model.pdb
$ python ../scripts/align_pdb.py --ref 7AAI_protein.pdb "7aai_protein_ligands/*/model.pdb" -o 7AAI_protein_ligands_af3.pdb # Align all predicted pdb files into one pdb file, in terms of a native protein structure. The output will be like this:
```
```text
📂 Loading reference: 7AAI_protein.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7AAI_protein.pdb
<mdtraj.Trajectory with 1 frames, 4626 atoms, 581 residues, and unitcells>
📂 Loading target: 7aai_protein_ligands/*/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein_ligands/seed-1_sample-0/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein_ligands/seed-1_sample-1/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein_ligands/seed-1_sample-2/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein_ligands/seed-1_sample-3/model.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/7aai_protein_ligands/seed-1_sample-4/model.pdb
<mdtraj.Trajectory with 5 frames, 4791 atoms, 589 residues, without unitcells>
🔍 Identifying protein backbone pocket atoms from reference...
⚠ Warning: No ligand found. Using protein backbone heavy atoms as pocket.
📐 Aligning target to reference using protein backbone pocket atoms...
📊 Pocket-Aligned Protein Backbone RMSD (MDTraj): [0.11138316 0.08106817 0.1130681  0.06714155 0.10324644] nm
📊 Pocket-Aligned Protein Backbone Pocket RMSD (MDTraj): [0.11138316 0.08106817 0.1130681  0.06714155 0.10324644] nm
💾 Saving aligned model structures
 - 7AAI_protein_ligands_af3.pdb
ℹ️ No ligand found — skipping ligand output.
✅ Alignment complete.
```

**Note**: We can see the predicted protein structures in the complex system have lower RMSD values. A potenital reason is that this system was included in the AF3 training data set. Its small RMSD values could not indicate its strong prediction capability. 


# More structural analysis

An example: PFOA - human serum albumin (hSA) protein
The goal of this example is to how we can use the AlphaFold 3 to predict the binding of PFOA with the hSA protein. 

**Background**

Maso, Lorenzo, et al. "Unveiling the binding mode of perfluorooctanoic acid to human serum albumin." Protein Science 30.4 (2021): 830-841. DOI: https://doi.org/10.1002/pro.4036

<img src="https://onlinelibrary.wiley.com/cms/asset/641b2e4e-b7a8-429b-8b78-d9238385a0ab/pro4036-fig-0001-m.jpg" alt="PFOA-hSA" width="1000">


**Figure 1**. Structure of hSA in complex with PFOA and Myr. Chemical structure (top) and composite omit maps depicting the (Fo−Fc) electron density (bottom) of PFOA (a) and Myr (b) contoured at 4σ; (c) Crystal structure of hSA-PFOA-Myr complex (white) obtained using a twofold molar excess of PFOA over Myr [PDB identification code: 7AAI]; (d) Superimposition of hSA-PFOA-Myr ternary complex (white) with aligned hSA-Myr binary complex (blue white) [PDB identification code: 7AAE]. The structure of hSA is organized in homologues domains (I, II and III), subdomains (A and B), fatty acids (FA) and Sudlow's binding sites. The α-helices of hSA are represented by cylinders. Bound PFOA and Myr are shown in a ball-and-stick representation with a semi-transparent van der Waals and colored by atom type (PFOA: carbon = dark salmon, oxygen = firebrick, fluorine = palecyan; Myr: carbon = smudge green, oxygen = firebrick). The electron density PFOA and Myr is shown as grey mesh. (Note: I switched the "7AAE" with "7AAI" after checking out both structures from the PDB database.)


## hSA-PFOA structural interactions

```bash
$ python ../scripts/get_interactions_from_pdb.py 7AAI.pdb 8PF
```
```text
[2025-11-06 13:29:54] Processing PDB file: 7AAI.pdb, Ligand: 8PF
[2025-11-06 13:29:54] Loading trajectory from 7AAI.pdb...
[2025-11-06 13:29:54] Trajectory loaded with 4966 atoms and 687 residues.
[2025-11-06 13:29:54] Using cutoff distance: 0.5 nm
[2025-11-06 13:29:54] Analyzing interactions for ligand residue ID: 581
# Identifying the residues within 0.5 nm of a ligand (chain_id: B, resSeq: 601)
# ------------------------------------------------------------------------------------
Non-Polar Residues:
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 488, resName: PHE), Min-Dist (8PF-F17,PHE-CB): 0.29 nm ***
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 387, resName: LEU), Min-Dist (8PF-O08,LEU-CD1): 0.31 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 423, resName: LEU), Min-Dist (8PF-F23,LEU-CD2): 0.50 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 430, resName: LEU), Min-Dist (8PF-F14,LEU-CD1): 0.32 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 453, resName: LEU), Min-Dist (8PF-F01,LEU-CD2): 0.33 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 415, resName: VAL), Min-Dist (8PF-F25,VAL-CG2): 0.38 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 491, resName: LEU), Min-Dist (8PF-F24,LEU-CD1): 0.38 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 460, resName: LEU), Min-Dist (8PF-F21,LEU-CD2): 0.31 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 473, resName: VAL), Min-Dist (8PF-F24,VAL-CG2): 0.47 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 426, resName: VAL), Min-Dist (8PF-F21,VAL-CG1): 0.40 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 457, resName: LEU), Min-Dist (8PF-F15,LEU-CD2): 0.33 nm
Polar Residues:
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 391, resName: ASN), Min-Dist (8PF-O08,ASN-ND2): 0.37 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 411, resName: TYR), Min-Dist (8PF-F12,TYR-OH): 0.25 nm ***
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 489, resName: SER), Min-Dist (8PF-O09,SER-OG): 0.25 nm ***
Positively Charged Residues:
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 414, resName: LYS), Min-Dist (8PF-F18,LYS-CD): 0.47 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 410, resName: ARG), Min-Dist (8PF-O08,ARG-NH2): 0.36 nm
  7AAI - Ligand (chain_id: B, resSeq: 601, resName: 8PF), Residue (chain_id: A, resSeq: 485, resName: ARG), Min-Dist (8PF-F03,ARG-CD): 0.35 nm
Negatively Charged Residues:
  N/A

For VMD visualization: (chain B and resname 8PF and resid 601) or (chain A and resid 488) or (chain A and resid 387) or (chain A and resid 391) or (chain A and resid 414) or (chain A and resid 423) or (chain A and resid 430) or (chain A and resid 453) or (chain A and resid 410) or (chain A and resid 415) or (chain A and resid 491) or (chain A and resid 460) or (chain A and resid 473) or (chain A and resid 411) or (chain A and resid 485) or (chain A and resid 489) or (chain A and resid 426) or (chain A and resid 457)

...

[2025-11-06 13:29:56] Processing complete.
```

## hSA-MYR structural interactions

```bash
$ python ../scripts/get_interactions_from_pdb.py 7AAI.pdb MYR
```
```text
[2025-11-06 13:32:05] Processing PDB file: 7AAI.pdb, Ligand: MYR
[2025-11-06 13:32:05] Loading trajectory from 7AAI.pdb...
[2025-11-06 13:32:05] Trajectory loaded with 4966 atoms and 687 residues.
[2025-11-06 13:32:05] Using cutoff distance: 0.5 nm
[2025-11-06 13:32:05] Analyzing interactions for ligand residue ID: 588
# Identifying the residues within 0.5 nm of a ligand (chain_id: I, resSeq: 608)
# ------------------------------------------------------------------------------------
Non-Polar Residues:
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 26, resName: ALA), Min-Dist (MYR-C11,ALA-CB): 0.42 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 152, resName: PRO), Min-Dist (MYR-C5,PRO-CG): 0.37 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 251, resName: LEU), Min-Dist (MYR-C9,LEU-CD2): 0.35 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 27, resName: PHE), Min-Dist (MYR-C13,PHE-CE2): 0.46 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 283, resName: LEU), Min-Dist (MYR-C4,LEU-CD2): 0.38 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 258, resName: ALA), Min-Dist (MYR-O1,ALA-N): 0.37 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 66, resName: LEU), Min-Dist (MYR-C14,LEU-CD2): 0.36 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 70, resName: PHE), Min-Dist (MYR-C13,PHE-CD1): 0.39 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 254, resName: ALA), Min-Dist (MYR-C2,ALA-O): 0.35 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 23, resName: VAL), Min-Dist (MYR-C13,VAL-CG2): 0.39 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 284, resName: LEU), Min-Dist (MYR-C3,LEU-CA): 0.40 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 46, resName: VAL), Min-Dist (MYR-C14,VAL-CG1): 0.45 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 250, resName: LEU), Min-Dist (MYR-C9,LEU-CD2): 0.41 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 22, resName: LEU), Min-Dist (MYR-C8,LEU-CD1): 0.41 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 14, resName: LEU), Min-Dist (MYR-C8,LEU-CD2): 0.43 nm
Polar Residues:
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 287, resName: SER), Min-Dist (MYR-O1,SER-OG): 0.23 nm ***
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 150, resName: TYR), Min-Dist (MYR-O2,TYR-OH): 0.27 nm ***
Positively Charged Residues:
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 10, resName: ARG), Min-Dist (MYR-C6,ARG-NH1): 0.41 nm
  7AAI - Ligand (chain_id: I, resSeq: 608, resName: MYR), Residue (chain_id: A, resSeq: 257, resName: ARG), Min-Dist (MYR-O2,ARG-NE): 0.29 nm ***
Negatively Charged Residues:
  N/A

For VMD visualization: (chain I and resname MYR and resid 608) or (chain A and resid 26) or (chain A and resid 152) or (chain A and resid 10) or (chain A and resid 251) or (chain A and resid 27) or (chain A and resid 283) or (chain A and resid 258) or (chain A and resid 66) or (chain A and resid 287) or (chain A and resid 70) or (chain A and resid 254) or (chain A and resid 23) or (chain A and resid 284) or (chain A and resid 46) or (chain A and resid 150) or (chain A and resid 257) or (chain A and resid 250) or (chain A and resid 22) or (chain A and resid 14)

...

[2025-11-06 13:32:08] Processing complete.

```

## Calculate the RMSD values

```bash
# Select the PFOA-1 molecule (resid 581) from the native pdb
$ python ../scripts/extract_pdb.py --ref 7AAI_protein_ligands.pdb --selection "protein or (resname '8PF' and resid 581)" -o x1.pdb

# Select the PFOA-1 molecule (resid 583) from AF3-predicted pdb
$ python ../scripts/extract_pdb.py --ref 7AAI_protein_ligands_af3.pdb --selection "protein or (resname '8PF' and resid 583 )" -o x2.pdb

# Calculate their RMSD value, and a RMSD < 0.2 nm means a good prediction
$ python ../scripts/check_rmsd.py --ref x1.pdb x2.pdb

```
```text
📂 Loading reference PDB: x1.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/x1.pdb
📂 Loading target PDB: x2.pdb
  - /home/xg69107/program/alphafold3/alphafold3_tutorial/test/x2.pdb
📊 Protein Backbone RMSD (Direct): Min = 0.067 nm ; [0.11137787 0.08108392 0.11306661 0.06710603 0.10325549] nm
📊 Protein Backbone RMSD (MDTraj): Min = 0.067 nm ; [0.11138316 0.08107853 0.11308297 0.06709146 0.10324644] nm
📊 Ligand RMSD (Direct): Min = 0.145 nm ; [0.21525969 0.1542886  0.16826949 0.22186017 0.1446819 ] nm
📊 Ligand RMSD (MDTraj): Min = 0.109 nm ; [0.16674478 0.10911302 0.14677718 0.17931049 0.12142784] nm
📊 Protein Backbone Pocket RMSD (Direct): Min = 0.048 nm ; [0.05840079 0.04779226 0.07242807 0.05053607 0.06331026] nm
📊 Protein Backbone Pocket RMSD (MDTraj): Min = 0.031 nm ; [0.03139924 0.03197302 0.03405008 0.03250873 0.03445952] nm
```
